## Example curve prediction script
In this notebook we will try to predict the function coefficients.

In [ ]:
%matplotlib widget

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras

from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import preprocessing

from generate_TF import GenerateTF, get_freq
from scipy.optimize import curve_fit
from prettytable import PrettyTable

from pytorchClassifiers import get_keras_nn, plot_history

avg_pool1d = keras.layers.AveragePooling1D



In [ ]:
# Load the data
df = pd.read_pickle('./data/tf-ampl-response-82000-noise0.1.pkl')
df.head()

In [ ]:
# Visualize the data
for index, row in df.sample(n=3).iterrows():
    y = row['amplitude']
    x = get_freq(row['fmax'], row['np'])
    plt.plot(x, y)
plt.show()

In [ ]:
# Extract the target variables
phase = df.pop('phase')
gain = df.pop('gain')

# All fmax, np should be equal
fmax = df.iloc[0].fmax
NP = df.iloc[0].np
df.drop(columns=['fmax', 'np'], inplace=True)

In [ ]:
# target_orig is the vector with the originale phase, gain labels
target_orig = np.array((phase, gain), dtype=np.float32).T

target_scaler = preprocessing.StandardScaler().fit(target_orig)
# target is scaled, better for training
target = target_scaler.transform(target_orig)


In [ ]:
# load the models
model1 = keras.models.load_model('models/keras/regression/phase_best')
model2 = keras.models.load_model('models/keras/regression/gain_best')
print(model1.summary())
print(model2.summary())


In [ ]:
def curve_fit_deluxe(func, freq, sample, trim_edges=0, kernel_size=1, stride=1, **kwargs):
    # center crop sample
    if trim_edges > 0:
        freq, sample = freq[trim_edges:-trim_edges], sample[trim_edges:-trim_edges]
    # prepare the shapes for avg_pooling
    freq = freq.reshape(1, -1, 1)
    sample = sample.reshape(1, -1, 1)
    # perform average pooling
    freq = avg_pool1d(pool_size=kernel_size, strides=stride)(freq).numpy().flatten()
    sample = avg_pool1d(pool_size=kernel_size, strides=stride)(sample).numpy().flatten()
    # pass to curve_fit
    return curve_fit(func, freq, sample, **kwargs)

In [ ]:
def classify(phase, gain):
    phase, gain = float(phase), float(gain)

    # table = ['Param', 'True', 'NeuralNet', 'Optimizer']
    table = []
    
    gen_tf_noise = GenerateTF(fb_attn_index=3, noise_amplitude=0.1, with_noise=True)
    x = gen_tf_noise.frequency

    gen_tf_no_noise = GenerateTF(fb_attn_index=3, with_noise=False)

    # I get one input with given phase, gain from X_validate or y_validate
    ampl, resp = gen_tf_noise(x, phase, gain, return_response=True)
    # ampl_scaled = torch.tensor(data_scaler.transform([ampl]), dtype=torch.float32)
    resp = np.array([resp.real, resp.imag]).T.reshape(1, -1, 2)

    # make predictions with the NN
    pred_phase = model1(resp,training=False).numpy()
    pred_gain = model2(resp, training=False).numpy()
    pred_scaled = np.array([pred_phase, pred_gain]).reshape(1, -1)

    # Descale them to get original values
    pred_descaled = target_scaler.inverse_transform(pred_scaled)
    pred_phase, pred_gain = float(pred_descaled[0, 0]), float(pred_descaled[0, 1])

    # Get optimizers results
    (opt_phase, opt_gain), _ = curve_fit_deluxe(gen_tf_no_noise, x, ampl, trim_edges=130,
                                                kernel_size=4, stride=1,
                                                bounds=([-20, 0.001], [20, 0.004]), method='trf')

    # I add them to the table

    table.append(['phase', np.round(phase, 2), np.round(pred_phase, 2), np.round(opt_phase, 2)])
    table.append(['gain', np.round(gain, 4), np.round(pred_gain, 4), np.round(opt_gain, 4)])

    # I plot them
    fig = plt.figure()
    p = plt.plot(x, gen_tf_no_noise(x, phase, gain), label=f'True', ls='-', color='black')
    plt.plot(x, gen_tf_noise(x, phase, gain), label=f'True, with Noise', ls='-', color='black', alpha=0.5)
    plt.plot(x, gen_tf_no_noise(x, pred_phase, pred_gain), label=f'NeuralNet',
             ls='--', color='tab:orange')
    plt.plot(x, gen_tf_no_noise(x, opt_phase, opt_gain), label=f'Optimizer',
             ls=':', color='tab:green')
    plt.xlabel('Frequency')
    plt.ylabel('Amplitude')
    plt.legend()
    # print(table.get_string())
    return fig, table

fig, table = classify(-15, 0.002)
print(table)


In [ ]:
import gradio as gr

# print(gain_encoder.classes_.astype(str).tolist())
# build the gradio interface
slider = gr.Slider(label=f'Phase', minimum=-20.,
                   maximum=20., step=1.)
dropdown = gr.Dropdown(label='Gain',
                       choices=np.round(np.geomspace(
                           0.001, 0.004, num=20, endpoint=True), 4).tolist(),
                       type="value")
interface = gr.Interface(fn=classify,
                         inputs=[slider,
                                 dropdown,
                                 ],
                         outputs=[
                             gr.Plot(label='Graphical Evaluation'),
                             gr.DataFrame(label='Numerical Evaluation',
                                          headers=['Param', 'True',
                                                   'NeuralNet', 'Optimizer'],
                                          datatype=['str', 'number',
                                                    'number', 'number'],
                                          row_count=2, col_count=(4, 'fixed')
                                          )
                         ],
                         interpretation="default",
                         )

interface.launch(debug=False)
